# **GETAROUND - ANALYSE D'IMPACT DES RETARDS DANS LA LOCATION DE VÉHICULES**
### **BLOC 06 : DÉPLOIEMENT**

---

### Importation des bibliothèques & Configuration

In [ ]:
# Manipulation de données
import pandas as pd
import numpy as np

# Visualisation
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt 
from plotly.subplots import make_subplots
import seaborn as sns

# Ignorer les avertissements (warnings) inutiles
import warnings
warnings.filterwarnings('ignore')

### **Note d'information**

- `df_...` : DataFrames utilisés pour l'analyse business

- `viz_...` : DataFrames utilisés pour les visuels de distribution (sans les outliers)

---

## **PARTIE 1 |  CHARGEMENT ET PRÉPARATION DES DONNÉES "DELAY"**

### 1. Chargement

In [4]:
# Chargement du dataset
df = pd.read_excel(r"get_around_delay_analysis.xlsx")
df_delay = df.copy()

# Aperçu des premières lignes
df_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [5]:
# Informations générales sur le dataframe
print(df_delay.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB
None


### 2. Préprocessing

In [6]:
# Convertir 'checkin_type' et 'state' en catégorie
df_delay['checkin_type'] = df_delay['checkin_type'].astype('category')
df_delay['state'] = df_delay['state'].astype('category')

In [7]:
# Renommer les colonnes se terminant par "_in_minutes" (plus concis)
df_delay = df_delay.rename(columns=lambda col: col.replace('_in_minutes', '') if col.endswith('_in_minutes') else col)

In [8]:
# Ajouter une colonne 'was_late' pour indiquer si le conducteur a eu un retard au check-out
df_delay['was_late'] = df_delay['delay_at_checkout'].apply(lambda x: np.nan if pd.isna(x) else x > 0)

In [9]:
# Informations générales et statistiques basiques de df_delay

print("\n========================= df_delay =========================\n")
print(df_delay.info())
print()
print(df_delay.describe(include='category'))
print()
df_delay.describe()


========================= df_delay =========================

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   rental_id                        21310 non-null  int64   
 1   car_id                           21310 non-null  int64   
 2   checkin_type                     21310 non-null  category
 3   state                            21310 non-null  category
 4   delay_at_checkout                16346 non-null  float64 
 5   previous_ended_rental_id         1841 non-null   float64 
 6   time_delta_with_previous_rental  1841 non-null   float64 
 7   was_late                         16346 non-null  object  
dtypes: category(2), float64(3), int64(2), object(1)
memory usage: 1.0+ MB
None

       checkin_type  state
count         21310  21310
unique            2      2
top          mobile  ended
freq      

,rental_id,car_id,delay_at_checkout,previous_ended_rental_id,time_delta_with_previous_rental
count,21310.000000,21310.000000,16346.000000,1841.000000,1841.000000
mean,549712.880338,350030.603426,59.701517,550127.411733,279.288430
std,13863.446964,58206.249765,1002.561635,13184.023111,254.594486
min,504806.000000,159250.000000,-22433.000000,505628.000000,0.000000
25%,540613.250000,317639.000000,-36.000000,540896.000000,60.000000
50%,550350.000000,368717.000000,9.000000,550567.000000,180.000000
75%,560468.500000,394928.000000,67.000000,560823.000000,540.000000
max,576401.000000,417675.000000,71084.000000,575053.000000,720.000000


### 3. Engineering (previous rental join)

In [10]:
# Identifier les cas problématiques liés au retard du conducteur précédent et le temps effectif d'attente du conducteur 

# Étapes :
# 1. Extraire les informations de retard au checkout de chaque location et les préparer pour être reliées à la location suivante.
# 2. Fusionner ces données avec le DataFrame principal en utilisant l'identifiant de la location précédente.
# 3. Créer un indicateur 'is_problematic' qui vaut True si le retard du conducteur précédent dépasse le délai prévu entre deux locations.
# 4. Calculer le temps d'attente effectif du client : si le retard du précédent conducteur est supérieur au délai prévu, on prend la différence, sinon le temps d'attente est nul.
# 5. Affiche les informations générales et les statistiques basiques

df_temp = df_delay.copy()
df_previous = df_temp[['rental_id', 'delay_at_checkout']].copy()
df_previous.columns = ['rental_id_prev', 'previous_delay_at_checkout'] 

df_join = df_temp.merge(df_previous, left_on='previous_ended_rental_id', right_on='rental_id_prev', how='left')
df_join['is_problematic'] = (df_join['previous_delay_at_checkout'] > df_join['time_delta_with_previous_rental'])
df_join['effective_wait'] = (df_join['previous_delay_at_checkout'] - df_join['time_delta_with_previous_rental']).clip(lower=0)

df_with_previous_rental = df_join.dropna(subset=['previous_delay_at_checkout'])

print("\n========================= df_with_previous_rental =========================\n")
print(df_with_previous_rental.info())
print()
print(df_with_previous_rental.describe(include='category'))
print()
df_with_previous_rental.describe()


========================= df_with_previous_rental =========================

<class 'pandas.core.frame.DataFrame'>
Index: 1729 entries, 6 to 21286
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   rental_id                        1729 non-null   int64   
 1   car_id                           1729 non-null   int64   
 2   checkin_type                     1729 non-null   category
 3   state                            1729 non-null   category
 4   delay_at_checkout                1476 non-null   float64 
 5   previous_ended_rental_id         1729 non-null   float64 
 6   time_delta_with_previous_rental  1729 non-null   float64 
 7   was_late                         1476 non-null   object  
 8   rental_id_prev                   1729 non-null   float64 
 9   previous_delay_at_checkout       1729 non-null   float64 
 10  is_problematic                   1729 non-null   bool    


,rental_id,car_id,delay_at_checkout,previous_ended_rental_id,time_delta_with_previous_rental,rental_id_prev,previous_delay_at_checkout,effective_wait
count,1729.000000,1729.000000,1476.000000,1729.000000,1729.000000,1729.000000,1729.000000,1729.000000
mean,552219.469636,351318.063042,28.134146,550008.854829,276.454598,550008.854829,-24.761712,22.388664
std,12877.503645,54354.135966,429.744537,13231.876109,254.228780,13231.876109,430.602411,333.087850
min,505560.000000,159533.000000,-1707.000000,505628.000000,0.000000,505628.000000,-4624.000000,0.000000
25%,543571.000000,324821.000000,-47.000000,540855.000000,60.000000,540855.000000,-54.000000,0.000000
50%,552290.000000,367632.000000,4.000000,550476.000000,180.000000,550476.000000,1.000000,0.000000
75%,562317.000000,390729.000000,54.000000,560727.000000,540.000000,560727.000000,44.000000,0.000000
max,576292.000000,415953.000000,9787.000000,575053.000000,720.000000,575053.000000,12968.000000,12548.000000


### 4. Traitement des outliers

In [11]:
# Préparation de DataFrames "VIZ" filtrés (sans les outliers) pour certaines visualisations (notamment les graphiques de distributions)
# Les graphiques doivent être compréhensible et facile d'accès pour le Product Manager (commanditaire).
# L'échelle logarithmique est utile pour le Data Scientist (pour voir les ordres de grandeur), mais pour un client business, elle est souvent illisible ou trompeuse. 

# Intervalle à garder (central 90%)
quantile_low = 0.05
quantile_high = 0.95
col_delay = 'delay_at_checkout'
limit_lower = df_delay[col_delay].quantile(quantile_low)
limit_upper = df_delay[col_delay].quantile(quantile_high)

# Filtrage sur les 2 DataFrames
viz_delay = df_delay[
    ((df_delay[col_delay] >= limit_lower) & 
     (df_delay[col_delay] <= limit_upper)) |
    (df_delay[col_delay].isna())
].copy()

viz_with_previous_rental = df_with_previous_rental[
    ((df_with_previous_rental[col_delay] >= limit_lower) &
     (df_with_previous_rental[col_delay] <= limit_upper)) | 
    (df_with_previous_rental[col_delay].isna())
].copy()

# Sous-ensemble
viz_ended = viz_delay[viz_delay['state'] == 'ended'].copy()
viz_with_previous_rental_ended = viz_with_previous_rental[viz_with_previous_rental['state'] == 'ended']
viz_late_rentals = viz_ended[viz_ended[col_delay] > 0].shape[0]

print("--- Informations concernant les graphiques de distibution ---")
print(f"Zoom sur les 90% centraux (intervalle {quantile_low:.0%} - {quantile_high:.0%})")
print(f"On garde les retours compris entre {limit_lower:.0f} min et {limit_upper:.0f} min")
nb_hidden = len(df_delay) - len(viz_delay)
nb_hidden_prev = len(df_with_previous_rental) - len(viz_with_previous_rental)
print(f"Nombre de valeurs extrêmes masquées : {nb_hidden} (sur {len(df_delay)})")
print(f"Nombre de valeurs extrêmes masquées pour les locations enchainées : {nb_hidden_prev} (sur {len(df_with_previous_rental)})")

--- Informations concernant les graphiques de distibution ---
Zoom sur les 90% centraux (intervalle 5% - 95%)
On garde les retours compris entre -230 min et 398 min
Nombre de valeurs extrêmes masquées : 1634 (sur 21310)
Nombre de valeurs extrêmes masquées pour les locations enchainées : 128 (sur 1729)


---

## **PARTIE 2 | VISUALISATIONS**

### 1. Distribution des retards au check-out (`delay_at_checkout`)

In [12]:
# Visualisation de la proportion des locations en retard 

df_pie = df_delay.dropna(subset=['delay_at_checkout']).copy()

# Comptage
late_status_counts = df_pie['was_late'].value_counts(normalize=True).reset_index()
late_status_counts.columns = ['Status', 'Proportion']
late_status_counts['Count'] = df_pie['was_late'].value_counts().values

# Mapping
late_status_counts['Status'] = late_status_counts['Status'].map({True: 'En retard', False: 'À l\'heure'})

# Création du label
late_status_counts['Label'] = late_status_counts.apply(
    lambda row: f"{row['Status']} : {row['Proportion']*100:.1f}% ({row['Count']})", # Arrondi à 1 décimale suffisant
    axis=1
)

# Création du graphique
fig_late_proportion = px.pie(
    late_status_counts,
    names='Label',
    values='Proportion',
    title='Proportion des retards (sur les check-outs documentés)',
    width=800,
    color='Status',
    color_discrete_map={'En retard':'#ef553b', 'À l\'heure':'#00CC96'},
    hole=0.4
)

# Annotation pour expliquer la base de calcul
fig_late_proportion.add_annotation(
    text=f"Base : {late_status_counts['Count'].sum()} locations.<br>(Exclut les locations annulées et les données manquantes)",
    x=0.5, y=-0.2, showarrow=False,
    font=dict(size=12, color="grey")
)

fig_late_proportion.show()

print(f"Temps de retard médian : {df_delay['delay_at_checkout'].median():.0f} minutes (sur l'ensemble des données)")
print(f"Temps de retard médian: {df_with_previous_rental['delay_at_checkout'].median():.0f} minutes (sur {len(df_with_previous_rental)} locations enchaînées)")
print(f"\nTemps de retard moyen : {df_delay['delay_at_checkout'].mean():.0f} minutes (sur l'ensemble des données)")
print(f"Temps de retard moyen: {df_with_previous_rental['delay_at_checkout'].mean():.0f} minutes (sur {len(df_with_previous_rental)} locations enchaînées)")

Temps de retard médian : 9 minutes (sur l'ensemble des données)
Temps de retard médian: 4 minutes (sur 1729 locations enchaînées)

Temps de retard moyen : 60 minutes (sur l'ensemble des données)
Temps de retard moyen: 28 minutes (sur 1729 locations enchaînées)


**Plus d'une location sur deux est en retard !**

Le **retard médian est divisé par 2 dans le cas de locations enchainées** (4 minutes contre 9 minutes sur l'ensemble des données).

*Remarque : en raison des valeurs extrêmes, les moyennes sont très élevées et moins pertinentes.*

In [13]:
# Visualisation de la distribution détaillée des retours (en avance, à l'heure, en retard) --> SANS LES OUTLIERS (zoom sur 96)

# Histogramme des retours (en avance, à l'heure et en retard)
fig_return = px.histogram(
    viz_ended,
    x='delay_at_checkout',
    nbins=100,
    title="Distribution des check-outs (en avance, à l'heure et en retard)",
    labels={'delay_at_checkout': 'Retour (minutes)'},
    color_discrete_sequence=['#ab63fa'],
    marginal="box", 
    height=500
)
fig_return.show()


# Visualisation de la distribution des retards pour voir les fréquences et identifier les retards typiques  --> SANS LES OUTLIERS

# Filtrer pour les retards
df_positive_delays = viz_ended[viz_ended['delay_at_checkout'] > 0]

# Histogramme des retards
fig_delay = px.histogram(
    df_positive_delays,
    x='delay_at_checkout',
    nbins=100,
    title='Distribution des retards au check-out (min)',
    labels={'delay_at_checkout': 'Retard (minutes)'},
    color_discrete_sequence=['#ab63fa'],
    marginal="box", 
    height=500
)
fig_delay.show()

### 2. Impact du type de check-in sur les retards : mobile *vs* connect (`checkin_type`)

* **Mobile** = contrat de location mobile sur applications natives (le conducteur et le propriétaire se rencontrent et signent tous deux le contrat de location sur le smartphone du propriétaire)
* **Connect** = le conducteur ne rencontre pas le propriétaire et ouvre la voiture avec son smartphone

In [14]:
# Proportion des types de check-in dans les locations

mobile_rentals = df_delay[df_delay['checkin_type'] == 'mobile'].shape[0]
connect_rentals = df_delay[df_delay['checkin_type'] == 'connect'].shape[0]
total_rentals = len(df_delay)

print(f"Locations avec rencontre entre le conducteur et le propriétaire (mobile) : {mobile_rentals} ({mobile_rentals/total_rentals:.1%})")
print(f"Locations avec ouverture sans clé et sans interaction physique avec le propriétaire (connect) : {connect_rentals} ({connect_rentals/total_rentals:.1%})")

Locations avec rencontre entre le conducteur et le propriétaire (mobile) : 17003 (79.8%)
Locations avec ouverture sans clé et sans interaction physique avec le propriétaire (connect) : 4307 (20.2%)


In [15]:
# Visualisation des retours par type de check-in (mobile vs connect)

# Préparation des données
df_checkin = df_delay.dropna(subset=['delay_at_checkout']).copy()
viz_checkin = viz_delay.dropna(subset=['delay_at_checkout']).copy()
median_rental_by_checkin = df_checkin.groupby('checkin_type', observed=True)['delay_at_checkout'].median().reset_index()

order_cat = ["mobile", "connect"]
colors = {'mobile': '#636EFA', 'connect': '#ab63fa'} 

# Création du subplot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Temps médian (retard ou en avance) par rapport à l'heure prévue au check-out (totalité des données)", "Distribution (zoom sur 90% des données)"),
    vertical_spacing=0.15,
    row_heights=[0.3, 0.7]
)

# Barplot (Moyenne)
fig.add_trace(
    go.Bar(
        x=median_rental_by_checkin['checkin_type'],
        y=median_rental_by_checkin['delay_at_checkout'],
        marker_color=[colors[t] for t in median_rental_by_checkin['checkin_type']],
        name="Moyenne",
        text=median_rental_by_checkin['delay_at_checkout'].round(1),
        textposition='auto'
    ),
    row=1, col=1
)

# Boxplot (Distribution)
for c_type in order_cat: # On boucle DANS L'ORDRE défini
    subset = viz_checkin[viz_checkin['checkin_type'] == c_type]
    
    fig.add_trace(
        go.Box(
            x=subset['checkin_type'],
            y=subset['delay_at_checkout'],
            name=c_type,
            marker_color=colors[c_type],
            boxpoints='outliers',
            jitter=0.3
        ),
        row=2, col=1
    )

# Layout avec ORDRE FORCÉ
fig.update_layout(
    height=900, width=1000,
    title_text="Comparatif Mobile vs Connect",
    showlegend=False,
    xaxis=dict(categoryorder='array', categoryarray=order_cat),
    xaxis2=dict(categoryorder='array', categoryarray=order_cat)
)

fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)
fig.show()


In [16]:
# Focus sur les retardataires seulement

# Couleurs par type de check-in
colors = {'mobile': '#636EFA', 'connect': '#ab63fa'} 
order_cat = ["mobile", "connect"]

# Filtrage des retards positifs
viz_only_late = viz_delay[viz_delay['delay_at_checkout'] > 0].copy()

# Création du subplot
fig_late = go.Figure()

# Ajout des boxplots par type de check-in
for c_type in order_cat:
    subset = viz_only_late[viz_only_late['checkin_type'] == c_type]
    
    fig_late.add_trace(
        go.Box(
            x=subset['checkin_type'],
            y=subset['delay_at_checkout'],
            name=c_type,
            marker_color=colors[c_type],
            boxpoints='outliers',
            jitter=0.3
        )
    )

# Mise à jour du layout
fig_late.update_layout(
    title="Distribution des retards positifs (zoom sur 90% des données)",
    xaxis=dict(categoryorder='array', categoryarray=order_cat),
    yaxis_title="Retard au check-out (minutes)",
    height=600,
    showlegend=False
)

fig.add_hline(y=0, line_dash="dash", line_color="black")

fig_late.show()

Le **retard médian est un peu plus intense (46 min) dans le cas des locations "mobile"** (contre 40 min pour les "connect")

### 3. Impact des retards sur les annulations (état de location = `state`)

In [17]:
# Distribution des états de location

df_state = df_delay.copy()

# Calcul des comptes et proportions
state_counts = df_state['state'].value_counts(normalize=True).reset_index()
state_counts.columns = ['State_Org', 'Proportion']

# Ajout du nombre absolu (Count)
state_counts['Count'] = df_state['state'].value_counts().values

# Traduction et Harmonisation
state_counts['État'] = state_counts['State_Org'].replace({
    "ended": "Terminé", 
    "canceled": "Annulé"
})

# Création du Label combiné
state_counts['Label'] = state_counts.apply(
    lambda row: f"{row['État']} : {row['Proportion']*100:.1f}% ({row['Count']})", 
    axis=1
)

# Graphique
fig_state_dist = px.pie(
    state_counts,
    names='Label',
    values='Proportion',
    title='Distribution globale des états de location',
    color='État',
    color_discrete_map={'Terminé':'#00CC96', 'Annulé':'#ef553b'}, 
    width=800,
    hole=0.4
)

# Note explicative
fig_state_dist.add_annotation(
    text=f"Total locations demandées : {state_counts['Count'].sum()}",
    x=0.5, y=-0.15, showarrow=False,
    font=dict(size=12, color="grey")
)

fig_state_dist.show()

La grande majorité des locations se terminent avec succès. \
Néanmoins, il existe une proportion non négligeable d'annulations (≈ 1/6).

Il sera intéressant de voir si ces annulations sont corrélées à des retards précédents ou à des délais trop courts entre deux locations.

In [18]:
# Taux d'annulation selon le délai court ou long (<2h ou >2h) entre locations

df_delay_delta = df_delay[df_delay['previous_ended_rental_id'].notna()].copy()
df_delay_delta["state"] = df_delay_delta["state"].replace({"ended": "Terminé", "canceled": "Annulé"})

threshold = 120

df_delay_delta['delta_category'] = df_delay_delta['time_delta_with_previous_rental'].apply(
    lambda x: 'Moins de 2h' if x < threshold else 'Plus de 2h'
)

cancel_rate = df_delay_delta.groupby('delta_category')['state'].value_counts(normalize=True).unstack()

print(f"--- Taux d'annulation selon le délai entre deux locations (<2h vs >2h) ---")
print(cancel_rate * 100)

--- Taux d'annulation selon le délai entre deux locations (<2h vs >2h) ---
state              Annulé    Terminé
delta_category                      
Moins de 2h     11.111111  88.888889
Plus de 2h      13.191489  86.808511


Contre-intuitivement, les **locations avec un délai de moins de 2h entre deux réservations sont légèrement moins annulées** (≈ 11 %) que celles avec un délai plus long (≈ 13 %). \
Les enchaînements serrés, considérés isolément, ne semblent pas constituer une cause principale d’annulation.

In [19]:
# Taux d'annulation selon le retard (> 60 minutes) du conducteur de la précédente location

df_delay_60min = df_delay[df_delay['previous_ended_rental_id'].notna()].copy()
df_delay_60min['60min_late'] = df_delay_60min['delay_at_checkout'] > 60

# Préparer une table de référence pour les locations précédentes ("previous")
df_previous_delay_60min = df_delay_60min[['rental_id', '60min_late']].copy()
df_previous_delay_60min.columns = ['rental_id_prev', 'was_60min_late_prev']

# Jointure magique (Self-Merge)
# On associe la colonne 'previous_ended_rental_id' (de la loc actuelle)
# à la colonne 'rental_id' (de la loc précédente)
df_merged_60min = df_delay_60min.merge(
    df_previous_delay_60min,
    left_on='previous_ended_rental_id',  # La référence dans la ligne actuelle
    right_on='rental_id_prev',           # L'identifiant de la ligne précédente
    how='inner'                          # On ne garde que ceux qui ont une location précédente
)

# Calculer le taux d'annulation
# On groupe par "Est-ce que le précédent était en retard ?"
analysis_60min_late = df_merged_60min.groupby('was_60min_late_prev')['state'].value_counts(normalize=True).unstack()

print("--- Taux d'annulation selon le retard (>60 min) de la location précédente ---")
print(analysis_60min_late * 100)

--- Taux d'annulation selon le retard (>60 min) de la location précédente ---
state                 canceled      ended
was_60min_late_prev                      
False                13.469388  86.530612
True                 15.384615  84.615385


Les **locations précédées d'un retard de plus d'1 heure sont légèrement plus annulées** (≈ 15,4%) que celles sans retard (≈ 13,5%). \
Bien que la différence soit faible, cela pourrait indiquer un léger impact du retard précédent sur le comportement des utilisateurs.

In [20]:
# Taux d'annulation selon le délai entre les locations (<2h ou >2h) ET le retard (>60 min) de la location précédente

threshold = 120

# Créer la colonne 'delta_category' 
df_merged_60min['delta_category'] = df_merged_60min['time_delta_with_previous_rental'].apply(
    lambda x: 'Moins de 2h' if x < threshold else 'Plus de 2h'
)

# Calculer le taux d'annulation pour chaque combinaison
cross_analysis_delay_delta = df_merged_60min.groupby(['delta_category', 'was_60min_late_prev'])['state'].value_counts(normalize=True).unstack()
cross_analysis_delay_delta = cross_analysis_delay_delta['canceled'] * 100  # Garder uniquement le taux d'annulation
cross_analysis_delay_delta = cross_analysis_delay_delta.unstack(level=0)  # Réorganiser pour plus de clarté

print("--- Taux d'annulation selon le délai entre locations ET le retard (>60 min) de la location précédente ---")
print(cross_analysis_delay_delta)

# Graphique

df_viz_cross = df_merged_60min.groupby(['delta_category', 'was_60min_late_prev'])['state'].value_counts(normalize=True).rename('proportion').reset_index()
df_viz_cross = df_viz_cross[df_viz_cross['state'] == 'canceled']

df_viz_cross['Retard Précédent'] = df_viz_cross['was_60min_late_prev'].map({
    True: 'Oui (> 60 min)', 
    False: 'Non (< 60 min)'
})

fig_cross = px.bar(
    df_viz_cross,
    x='delta_category',
    y='proportion',
    color='Retard Précédent',
    barmode='group', # Pour mettre les barres côte à côte
    title="Explosion des annulations : Cumul délai court + Retard",
    labels={
        'delta_category': 'Temps de battement prévu',
        'proportion': "Taux d'annulation"
    },
    color_discrete_map={'Oui (> 60 min)': '#ef553b', 'Non (< 60 min)': '#00cc96'},
    text_auto='.1%' # Affiche le pourcentage sur les barres
)

fig_cross.update_layout(yaxis_tickformat='.0%') # Format axe Y en %

fig_cross.show()

--- Taux d'annulation selon le délai entre locations ET le retard (>60 min) de la location précédente ---
delta_category       Moins de 2h  Plus de 2h
was_60min_late_prev                         
False                  12.244898   14.285714
True                   22.222222   13.333333


**Effet combiné : lorsque les locations s’enchaînent à moins de 2 heures et qu’un retard dépasse 60 minutes, le taux d’annulation grimpe à 22%.** \
Cette configuration représente probablement un point de friction. \
Pour les délais > 2h entre les locations, le retard de la location précédente influence peu le taux d'annulation.

### 4. Distribution des délais inter-locations ou battements (`time_delta_with_previous_rental`)

In [21]:
# Visualisation de la distribution des battements
viz_gap = viz_with_previous_rental_ended.dropna(subset=['time_delta_with_previous_rental']).copy()

fig_gap = px.histogram(
    viz_gap,
    x='time_delta_with_previous_rental',
    nbins=30,
    title="Distribution des délais inter-locations (zoom sur 90% des données)",
    labels={'time_delta_with_previous_rental': 'battement (minutes)'},
    color_discrete_sequence=['#ab63fa'],
    marginal="box", 
    height=500
)
fig_gap.show()

print(f"Temps de battement médian (locations enchainées): {round(df_delay['time_delta_with_previous_rental'].median())} minutes")
print(f"Temps de battement moyen (locations enchainées): {round(df_delay['time_delta_with_previous_rental'].mean())} minutes")

Temps de battement médian (locations enchainées): 180 minutes
Temps de battement moyen (locations enchainées): 279 minutes


### 5. Cas problématiques (`is_problematic`)

In [22]:
# Visualisation des cas problématiques (frictions)

df_pie_prob = df_with_previous_rental.dropna(subset=['is_problematic']).copy()

# Calcul des proportions et des effectifs
problem_status_counts = df_pie_prob['is_problematic'].value_counts(normalize=True).reset_index()
problem_status_counts.columns = ['Status', 'Proportion']

# Ajout du nombre absolu (Count) pour vérification
problem_status_counts['Count'] = df_pie_prob['is_problematic'].value_counts().values

# Mapping des noms
problem_status_counts['Status'] = problem_status_counts['Status'].map({
    True: 'Cas problématique (friction)', 
    False: 'Sans impact'
})

# Création du Label combiné pour la légende
problem_status_counts['Label'] = problem_status_counts.apply(
    lambda row: f"{row['Status']} : {row['Proportion']*100:.1f}% ({row['Count']})", 
    axis=1
)

# Création du graphique
fig_problematic_proportion = px.pie(
    problem_status_counts,
    names='Label',
    values='Proportion',
    title='Impact réel : proportion des cas problématiques',
    width=800,
    color='Status',
    color_discrete_map={'Cas problématique (friction)':'#ef553b', 'Sans impact':'#00CC96'},
    hole=0.4
)

# Annotation pour donner le contexte
total_cases = problem_status_counts['Count'].sum()
fig_problematic_proportion.add_annotation(
    text=f"Base : {total_cases} locations enchaînées (avec location précédente proche).<br>Un 'Cas problématique' est défini par : Retard précédent > Temps de battement.",
    x=0.5, y=-0.2, showarrow=False,
    font=dict(size=12, color="grey")
)

fig_problematic_proportion.show()

mean_delay = df_with_previous_rental.loc[df_with_previous_rental['is_problematic'], 'delay_at_checkout'].mean()
print(f"Moyenne du retard dans les cas problématiques : {mean_delay:.0f} minutes")

median_delay = df_with_previous_rental.loc[df_with_previous_rental['is_problematic'], 'delay_at_checkout'].median()
print(f"Médiane du retard dans les cas problématiques : {median_delay:.0f} minutes")

nb_cancelled = df_with_previous_rental[(df_with_previous_rental['is_problematic']) & (df_with_previous_rental['state'] == 'canceled')].shape[0]
print(f"Nombre d'annulation dans les cas problématiques' : {nb_cancelled}")


Moyenne du retard dans les cas problématiques : 118 minutes
Médiane du retard dans les cas problématiques : 24 minutes
Nombre d'annulation dans les cas problématiques' : 37


Les annulations s'élèvent à 16 % dans les cas problématiques. \
*Pour rappel : 15 % d'annulation sur l'ensemble des données*

In [23]:
# Visualisation de la relation entre les retours au check-out et les délais inter-locations

viz_plot = viz_with_previous_rental_ended.dropna(
    subset=['time_delta_with_previous_rental', 'previous_delay_at_checkout', 'delay_at_checkout']
).copy()

# Création du graphique
fig_friction_zone = px.scatter(
    viz_plot,
    x='time_delta_with_previous_rental', # AXE X : Le temps prévu
    y='previous_delay_at_checkout',      # AXE Y : Le retard du PRECEDENT (La source du problème)
    color='is_problematic',
    title='Zone de friction | Retard précédent vs Temps de battement (zoom de 0 à 10h)',
    labels={
        'time_delta_with_previous_rental': 'Temps de battement prévu (min)',
        'previous_delay_at_checkout': 'Retard du conducteur PRÉCÉDENT (min)',
        'is_problematic': 'Impact sur le suivant',
        'delay_at_checkout': 'Retard du conducteur ACTUEL' # Pour l'info-bulle
    },
    hover_data=['rental_id', 'delay_at_checkout'], 
    opacity=0.6,
    height=600,
    color_discrete_map={True: '#ef553b', False: '#00CC96'} # Rouge / Vert
)

# Ajout de la ligne de sécurité (diagonale)
max_val = 800 # On limite un peu la ligne pour l'esthétique
fig_friction_zone.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    name='Limite critique (Retard = Battement)',
    line=dict(color='orange', width=2, dash='dash')
))

# Zoom
fig_friction_zone.update_xaxes(range=[-5, 610])
fig_friction_zone.update_yaxes(range=[-200, 650])

fig_friction_zone.show()

🔴 **Zone de friction (rouge, au-dessus de la ligne)** : le retard a dépassé le temps de battement. Le client suivant a dû attendre.

🟢 **Zone sécurisée (vert, en dessous de la ligne)** : le battement était suffisant pour absorber le retard (ou le conducteur était en avance/à l'heure).

L'objectif du seuil (threshold) sera d'interdire les locations avec un délai prévu trop court (partie gauche du graphique). \
Cela éliminera mécaniquement les points rouges situés dans cette zone, mais "sacrifiera" aussi les points verts qui s'y trouvent. \
C'est ce compromis que nous devons maintenant chiffrer.

---

## **PARTIE 3 | THRESHOLD SIMULATION**

### Coût (perte de volume d'affaires) *vs* Bénéfice (réduction des frictions client) pour différents seuils

In [24]:
# Visualisation du nombre de cas problématiques résolus et du nombre de locations affectées en fonction du seuil  de délai minimum


# --- 1. Préparation des Données ---

# Pour le GAIN (Problèmes résolus)
df_gain = df_join.dropna(subset=['is_problematic', 'time_delta_with_previous_rental']).copy()

# Pour le COÛT (Locations perdues)
df_cost = df_join.dropna(subset=['time_delta_with_previous_rental']).copy()


# --- 2. La Boucle de simulation ---

# On se limite à 5h (300 min) car au-delà l'impact est minime et ça écrase le graph
thresholds = range(0, 301, 30) 

results = []

for t in thresholds:
    # MÉTRIQUE 1 : Combien de problèmes on résout ?
    # (Ceux qui étaient problématiques ET qui ont un délai < seuil)
    solved = df_gain[
        (df_gain['is_problematic'] == True) & 
        (df_gain['time_delta_with_previous_rental'] < t)
    ].shape[0]
    
    # MÉTRIQUE 2 : Combien de locations on perd ?
    # (Toutes celles qui ont un délai < seuil, peu importe leur état)
    lost = df_cost[
        df_cost['time_delta_with_previous_rental'] < t
    ].shape[0]
    
    results.append({
        'Seuil': t,
        'Problèmes résolus': solved,
        'Locations perdues': lost
    })

# Création du DataFrame de résultats
df_sim = pd.DataFrame(results)


# --- 3. Graphique de compromis (Trade-off) ---

fig_tradeoff = go.Figure()

# Courbe des pertes (coût)
fig_tradeoff.add_trace(go.Scatter(
    x=df_sim['Seuil'], 
    y=df_sim['Locations perdues'],
    mode='lines+markers',
    name='Locations perdues (coût)',
    line=dict(color='#ef553b', width=3)
))

# Courbe des gains (bénéfice)
fig_tradeoff.add_trace(go.Scatter(
    x=df_sim['Seuil'], 
    y=df_sim['Problèmes résolus'],
    mode='lines+markers',
    name='Problèmes résolus (Gain)',
    line=dict(color='#00cc96', width=3)
))

# Mise en forme
fig_tradeoff.update_layout(
    title="Simulation de l'impact : compromis Coût / Bénéfice",
    xaxis_title="Seuil de délai minimum (minutes)",
    yaxis_title="Nombre de locations",
    legend=dict(x=0.01, y=0.99), # Légende à l'intérieur pour gagner de la place
    hovermode="x unified",        # Affiche les deux valeurs en même temps au survol
    height=800
)

# Annotation
fig_tradeoff.add_annotation(
    text="L'écart entre les courbes représente<br>la 'rentabilité' de la mesure.",
    xref="paper", yref="paper",
    x=0.5, y=0.5, showarrow=False,
    font=dict(color="grey", size=12)
)

fig_tradeoff.show()


##############################################################################################


# Visualisation de l'impact sur le chiffre d'affaires global (% volume préservé)


# --- 1. Préparation des données ---

df_total = df_delay.copy() 
scopes = ['mobile', 'connect']
results_by_scope = {}


# --- 2. Boucle de Simulation ---

thresholds = range(0, 301, 30)

for scope in scopes:
    # Filtrer les sous-ensembles pour ce scope
    d_gain_scope = df_gain[df_gain['checkin_type'] == scope]
    d_cost_scope = df_cost[df_cost['checkin_type'] == scope]
    
    # Calculer le volume total
    total_business_volume = len(df_total[df_total['checkin_type'] == scope])
    
    scope_results = []
    
    for t in thresholds:
        # Gain (Problèmes résolus)
        solved = d_gain_scope[
            (d_gain_scope['is_problematic'] == True) & 
            (d_gain_scope['time_delta_with_previous_rental'] < t)
        ].shape[0]
        
        # Coût (Locations enchaînées perdues)
        lost = d_cost_scope[
            d_cost_scope['time_delta_with_previous_rental'] < t
        ].shape[0]
        
        scope_results.append({
            'Seuil': t,
            'Problèmes résolus': solved,
            'Locations perdues': lost,
            '% Volume préservé': ((total_business_volume - lost) / total_business_volume) * 100
        })
    
    results_by_scope[scope] = pd.DataFrame(scope_results)


# --- 3. Visualisation ---

fig_diff = make_subplots(
    rows=1, cols=2, 
    subplot_titles=("Scope : Mobile", "Scope : Connect"),
    shared_yaxes=False # Axes Y libres car les volumes sont très différents
)

for i, scope in enumerate(scopes):
    df_res = results_by_scope[scope]
    col_idx = i + 1
    
    # Courbe VOLUME PRÉSERVÉ
    fig_diff.add_trace(go.Scatter(
        x=df_res['Seuil'], y=df_res['% Volume préservé'],
        mode='lines', name=f'% préservé {scope}',
        line=dict(color='#636EFA', width=3),
        legendgroup='vol'
    ), row=1, col=col_idx)

    # Ligne rouge de "Limite acceptable" (ex: 90%)
    fig_diff.add_hline(y=90, line_dash="dot", line_color="red", row=1, col=col_idx, 
                       annotation_text="Obj: 90%", annotation_position="bottom right")

fig_diff.update_layout(
    title="Impact sur le chiffre d'affaires global (% volume préservé)",
    yaxis_title="% des locations maintenues",
    xaxis_title="Seuil (minutes)",
    height=500, width=1000,
    showlegend=False
)
# Axe Y forcé à démarrer à 80% 
fig_diff.update_yaxes(range=[80, 101]) 

fig_diff.show()

**Efficacité immédiate :** \
Il y a beaucoup de "frictions inutiles" sur des délais très courts. \
L'introduction d'un seuil, même faible (30 min), permet de résoudre une part significative des problèmes pour un impact minime sur le chiffre d'affaires.

**Point d'inflexion ("Sweet Spot") :** \
Au-delà de 120-150 minutes, le nombre de locations perdues (courbe rouge) continue de croître linéairement, tandis que le nombre de problèmes résolus (courbe verte) plafonne.

---

## **PARTIE 4 | CONCLUSION & RECOMMANDATIONS**

**Recommandations sur le périmètre (Scope) :**
- Pour les **voitures "Mobile"** qui nécessitent une rencontre physique (signature de contrat, état des lieux), le risque de conflit et de retard est plus grand. \
Conseil : appliquer un **seuil haut (ex: 90 ou 120 minutes)** car ce flux est plus sujet aux aléas et aux frictions interpersonnelles. \
Un retard ici étant particulièrement problématique pour l'expérience utilisateur, un tampon de sécurité plus large est nécessaire car il est préférable de privilégier la sérénité et la qualité de service.
- Pour les **voitures "Connect"** qui s'ouvrent sans clé, les retards sont souvent moindres et la gestion est plus fluide. \
Conseil : appliquer un **seuil plus bas (ex: 30 ou 60 minutes)** pour maximiser la disponibilité des véhicules et privilégier le revenu car la friction est moindre.


**Ce compromis permettrait d'éliminer la majorité des cas critiques tout en préservant plus de 95% du volume total de locations.** \

***N.B. :***

- *Les locations "isolées" sont sécurisées par définition (le seuil ne les touchera jamais car elles n'ont pas de location précédente proche).* \

- *Les voitures "Connect" sont beaucoup plus souvent louées "à la chaîne" (back-to-back) que les "Mobile".* \
*Elles représentent 45% dans le volume de locations enchaînées contre 20% dans le volume total.*

- *La courbe "Mobile" reste très haute (au-dessus de 96%) même avec un seuil de 300 min, car la grande majorité des locations mobiles ne sont pas enchaînées et donc pas impactées.* \
*Elles ont plus souvent des "trous" dans leur planning (pas de `time_delta`).*